#### Names of people in the group

Please write the names of the people in your group in the next cell.

Marcus Stephan Nordal

Silviu Mihai

In [ ]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-acb044fd-074b-4c09-a992-f4a672226a5e/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Loading modules that we need
from pyspark.sql.dataframe import DataFrame
from collections import Counter
from pyspark.sql.functions import desc

In [ ]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
posts_df = load_df("/user/hive/warehouse/posts")

#### Subtask 1: implementing two functions
Implement these two functions:
1. 'compute_pearsons_r' that receives a DataFrame and two column names and returns the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) between values of two columns;
2. 'make_tag_graph' that in the input receives the DataFrame containing the records related to 'questions' and returns a DataFrame with two columns 'u' and 'v'; the record for row i from the resulting DataFrame is a tuple (u_i, v_i). u_i and v_j are distinct tags and have appeared together for a question.

Please note that you should implement the 'compute_pearsons_r' yourself, so you should not use the 'DataFrame.stat.corr' method. Nevertheless, you can use 'DataFrame.stat.corr' to verify the correctness of your implementation.

In [ ]:
from pyspark.sql.functions import split, explode, col, trim
from pyspark.sql import functions as Func
from pyspark.sql import SparkSession

def compute_pearsons_r(df: "a DataFrame", col1: "name of column A", col2: "name of column B") -> float:
    mean1 = df.agg(Func.mean(col1)).first()[0]
    mean2 = df.agg(Func.mean(col2)).first()[0]

    squared_deviation1 = (Func.sum(Func.pow(Func.col(col1) - mean1, 2)))
    squared_deviation2 = (Func.sum(Func.pow(Func.col(col2) - mean2, 2)))
    value_count1 = (Func.count(Func.col(col1)) - 1)
    value_count2 = (Func.count(Func.col(col2)) - 1)

    variance1 = df.agg(squared_deviation1 / value_count1).first()[0]
    variance2 = df.agg(squared_deviation2 / value_count2).first()[0]
    col_product = (Func.sum(Func.col(col1) * Func.col(col2)))
    
    covariance = df.agg(col_product / value_count1).first()[0] - mean1 * mean2

    return covariance / ((variance1 ** 0.5) * (variance2 ** 0.5))

##Test
result = compute_pearsons_r(users_df, "Reputation", "UpVotes")
print(result)

def make_tag_graph(df: "DataFrame containing question data") -> DataFrame:
    questions_df = df.filter(col("PostTypeId") == 1)

    questions_df = questions_df.withColumn("Tags", Func.regexp_replace("Tags", "<(.*?)>", "$1,"))
    questions_df = questions_df.withColumn("Tags", Func.regexp_replace("Tags", ">(.*?)<", "$1,"))
    print("After removing the < >:")
    questions_df.show()

    questions_df = questions_df.withColumn("Tags", explode(split(col("Tags"), ",")))
    questions_df = questions_df.withColumn("Tags", trim(col("Tags")))
    questions_df = questions_df.filter(col("Tags").isNotNull() & (col("Tags") != ""))
    
    print("After splitting and exploding & trimming:")
    questions_df.show()

    result_df = (
        questions_df.select(col("Tags").alias("u"))
        .crossJoin(
            questions_df.select(col("Tags").alias("v"))
        )
        .where((col("u").isNotNull()) & (col("u") != "") & (col("v").isNotNull()) & (col("v") != ""))
        .distinct()
    ).distinct()

    result_df = result_df.dropDuplicates()

    print("Data after crossjoining tags")
    result_df.show()

    df_pandas = result_df.toPandas()
    print(df_pandas)

    return result_df

result = make_tag_graph(posts_df)

##Show result
result.show()

0.5218109904695661
Before cross join:
+---+--------+----------+------------+-----+---------+--------------------+-----------+-------------------+--------------------+----------------+-----------+------------+-------------+-------------------+
| Id|ParentId|PostTypeId|CreationDate|Score|ViewCount|                Body|OwnerUserId|   LastActivityDate|               Title|            Tags|AnswerCount|CommentCount|FavoriteCount|          CloseDate|
+---+--------+----------+------------+-----+---------+--------------------+-----------+-------------------+--------------------+----------------+-----------+------------+-------------+-------------------+
|  5|    null|         1|        null|    9|      789|PHA+SSd2ZSBhbHdhe...|          5|2014-05-14 00:36:31|SG93IGNhbiBJIGRvI...|machine-learning|          1|           1|            1|2014-05-14 14:40:25|
|  7|    null|         1|        null|    4|      459|PHA+QXMgYSByZXNlY...|         36|2014-05-16 13:45:00|V2hhdCBvcGVuLXNvd...|       educati

#### Subtask 2: implementing three functions
Impelment these three functions:
1. 'get_nodes' that, given the result from execution of 'make_tag_graph', returns a DataFrame with one column named 'id' that includes the tags that have appeared in the tag graph;
2. 'get_edges' that, given the result from execution of 'make_tag_graph', returns a DataFrame with two columns 'src' and 'dst' where 'src' is the source node and 'dst' is the destination node.


Note that the term 'tag graph' in this context refers to the DataFrame reuturned by executing 'make_tag_graph'. Furthermore, 'src' and 'dst' are distinct, so 'src' != 'dst'.

In [ ]:
def get_nodes(df: "DataFrame of the tag graph") -> DataFrame:
    nodes_df = df.select(col("u")).union(df.select(col("v"))).distinct()
    nodes_df = nodes_df.withColumnRenamed("u", "id")
    return nodes_df
  

def get_edges(df: "DataFrame of the tag graph") -> DataFrame:
  edges_df = df.select(col("u").alias("src"), col("v").alias("dst"))
  #
  return edges_df



In [ ]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


#### Subtask 3: validating the implementation by running the tests

Run the cell below and make sure that all the tests run successfully.

In [ ]:
%%unittest_main
class TestTask3(unittest.TestCase):
  
  error_threshold = 0.03
  
  def test_corr1(self):
    # Pearson correlation coefficient between 'user reputation' and 'upvotes' received by users
    result = compute_pearsons_r(users_df, "Reputation", "UpVotes")
    self.assertLessEqual(abs(result-0.5218138310114108), self.error_threshold)
    print(result)
  
  def test_corr2(self):
    # Pearson correlation coefficient between 'user reputation' and 'downvotes' received by users
    result = compute_pearsons_r(users_df, "Reputation", "DownVotes")
    self.assertLessEqual(abs(result-0.1473558141546844), self.error_threshold)
    print(result)

  def test_corr3(self):
    # Pearson correlation coefficient between 'question score' and the 'number of answers' it received
    result = compute_pearsons_r(posts_df[posts_df["PostTypeId"] == 1], "Score", "AnswerCount")
    self.assertLessEqual(abs(result-0.47855272641249674), self.error_threshold)
    print(result)
    
  def test_make_tag_graph(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    self.assertIsInstance(result, DataFrame)
    
    coulmn_names = Counter(map(str.lower, ['u', 'v']))
    self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)), "Missing column(s) or column name mismatch")
    
    display(result)
    
    self.assertEqual(result.count(), 228830)
    
  def test_get_nodes(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    n = get_nodes(result)
    self.assertEqual(n.count(), 638)
    n.show()

  def test_get_edges(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    e = get_edges(result)
    
    coulmn_names = Counter(map(str.lower, ['src', 'dst']))
    self.assertCountEqual(coulmn_names, Counter(map(str.lower, e.columns)), "Missing column(s) or column name mismatch")
    
    self.assertEqual(e.count(), 225290)
    e.show()
    

0.5218109904695661
0.14735501007545418
0.4753227874233796


#### Subtask 4: answering to questions about Spark related concepts

Please write a short description for the terms below---one to two short paragraphs for each term. Don't copy-paste; instead, write your own understanding.

1. What do the terms 'User-Defined Functions (UDFs)', 'Data Locality', 'Bucketing', 'Distributed Filesystem' mean in the context of Spark?

Write your descriptions in the next cell.

Your answers...